In [1]:
%load_ext autoreload
%autoreload 2
%gui qt

In [2]:
import os
import pya

TEST_FILE = os.path.join(os.path.abspath('..'), "data\\stereoTest.wav")
signal = pya.Asig(TEST_FILE)
signal = signal[:,0]

In [3]:
import numpy as np
from pyavis.backends.qt_v2 import TrackQt, AxisQt
from pyavis.backends.qt_v2.multitrack import MultiTrackQt

mt = MultiTrackQt()
track = mt.add_track("test_1", 44100)

mt.show()

HEllo


In [4]:
def midi_conv(value) -> str:
    midi_value = int(value)
    notes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B',]
    octave = int(midi_value / 12 - 1)
    return f'{notes[midi_value % 12]}{octave}'

ax_left = AxisQt("left", lambda value: f'{round(value,1)} | {midi_conv(value)}')
ax_left.set_spacing((5,1))
ax_bottom = AxisQt("bottom", lambda value: f'{round(value / track.sampling_rate, 2)}') 

In [5]:
track.set_axis(ax_left)
track.set_axis(ax_bottom)

In [6]:
sig = track.add_signal((0,25), signal.sig)
sig.set_size(10)

rect = track.add_rectangle((0,20), (len(sig.data), 10))
rect.set_style(style={"fill": (0,255,0, 25), "border": (255,0,0, 255)})

In [7]:
track.set_view_limits(xMin = 0, xMax=len(sig.data), yMin = 0, yMax=130)

In [8]:
def move_sig(signal, args):
    sig.set_position(args[1][0], args[1][1] + 5)

def print_pitch(value):
    print(round(440 * np.power(2,(value - 69) / 12), 1))

rect.positionChanged.connect(lambda args: move_sig(sig, args))
rect.positionChanged.connect(lambda args: print_pitch(args[1][1]))

In [9]:
rect.draggable = True
track.allow_dragging(False, True)
track.set_dragging_limits((0, len(sig.data)), ((1, 128)))

30.4
31.1
31.4
32.9
34.4
35.6
37.2
38.0
38.3
39.8
41.1
42.1
44.1
46.2
47.7
48.7
50.3
53.1
55.2
57.0
59.2
60.1
59.9
59.4
54.6
48.9
43.5
38.4
35.3
34.0
32.2
30.0
28.2
26.5
26.3
25.9
26.2
26.4
28.1
30.4
32.9
33.8
38.3
39.0
39.0
39.3
39.3
56.6
62.7
88.9
121.2
154.9
190.4
223.1
299.2
376.6
470.2
587.0
738.8
922.5
1142.7
1349.8
1607.0
1943.9
2242.1
2485.5
2912.7
3228.9
3694.9
4261.8
4577.0
4876.8
5155.1
5321.2
5714.8
6041.0
6436.6
7023.3
7135.5
7542.8
7973.3
8361.8
8495.5
8839.1
8839.1
8980.4
9417.9
10276.3
10691.9
11666.3
12234.8
12430.4
12430.4
12933.1
13139.8
13289.8
13289.8
13289.8
13289.8
13289.8
13289.8
13289.8
13289.8
13289.8
13289.8
13289.8
13244.4
13036.1
12332.2
11947.2
11759.2
11302.1
10777.0
10358.1
9876.8
8909.4
8563.1
8563.1
10034.7
9343.5
7483.2
6385.8
5449.3
4915.6
4364.3
3936.9
3551.3
3254.6
2982.8
2777.3
2586.0
2259.9
1990.7
1795.7
1685.3
1508.2
1415.5
1339.1
1207.9
1089.6
929.8
852.2
704.5
615.6
497.0
420.8
376.6
323.9
257.4
191.9
156.2
129.1
102.6
82.2
70.1
58.9
50.6
43.2